In [2]:
import csv
import boto3
from kafka import KafkaConsumer

In [3]:
aws_access_key_id = ''
aws_secret_access_key = ''
aws_region = 'us-west-2'
bucket_name = 'zillowlakersbucket'
folder_name = 'kafka-read/'

In [4]:
consumer = KafkaConsumer(
    'lakers-zillow',  
    bootstrap_servers='34.215.164.92:9092', 
    group_id='lakers-zillow',  
    enable_auto_commit=False  
)

In [5]:
# Create an S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region
)

In [5]:
# import s3fs
# import pandas as pd

# csv_file_path = 'supermarket_sales - Sheet1.csv'
# df = pd.read_csv(csv_file_path)

# s3 = s3fs.S3FileSystem(anon=False)

# csv_filename = "now_sales.csv"

# with s3.open(f"{bucket_name}/{csv_filename}",'w') as f:
#     df.to_csv(f)

In [6]:
# Iterate over Kafka messages
for message in consumer:
    key = message.key.decode('utf-8')
    value = message.value.decode('utf-8')
    csv_data = [[key, value]]
    csv_file_key = folder_name + 'lakers-zillow.csv'
    csv_rows = csv_data.splitlines()
    csv_file = csv.reader(csv_rows)
    csv_filename = "lakers-zillow.csv"  
    
    # Create the S3 key (file path)
    s3_key = f"{folder_name}/{csv_filename}" if folder_name else csv_filename
    
    csv_string = "\n".join(csv_rows)
    
    # Upload CSV file to S3
    s3.put_object(
        Bucket=bucket_name,
        Key=s3_key,
        Body=csv_string,
    )

KeyboardInterrupt: 

In [ ]:
#Commit the Kafka offset manually after successful processing
consumer.commit()